In [6]:
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt

client = MongoClient('localhost', 27017)  
db = client.project_3  # Change 'project_3' to your actual database name
collection = db.fridge_fills  # Change 'your_collection_name' to your actual collection name

cursor = collection.find({})  # You can add query filters here if needed

data = list(cursor)

df = pd.DataFrame(data)

df.drop('_id', axis=1, inplace=True)

df.head()


,Date,Day of Week,Time of Day,Name,Pickup Location,Dropoff Location,Fill Type
0,2023-01-01 05:00:00,Sunday,Morning (7AM-12PM),Krista Kircanski,GO - Grocery Outlet,BOK,Rescued Groceries
1,2023-01-02 05:00:00,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,BOK,Purchased Groceries
2,2023-01-02 05:00:00,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,DS - DiSilvestro,Purchased Groceries
3,2023-01-02 05:00:00,Monday,Evening (5PM-10PM),Kathryn Nolan,Grocery FILL,BOK,Purchased Groceries
4,2023-01-02 05:00:00,Monday,Morning (7AM-12PM),Leah Santos,Grocery FILL,Mifflin,Purchased Groceries


In [2]:
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x).date())
df_dates = df

# Display the DataFrame
df_dates.head()

,Date,Day of Week,Time of Day,Name,Pickup Location,Dropoff Location,Fill Type
0,2023-01-01,Sunday,Morning (7AM-12PM),Krista Kircanski,GO - Grocery Outlet,BOK,Rescued Groceries
1,2023-01-02,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,BOK,Purchased Groceries
2,2023-01-02,Monday,Evening (5PM-10PM),Kate Gilbert,Grocery FILL,DS - DiSilvestro,Purchased Groceries
3,2023-01-02,Monday,Evening (5PM-10PM),Kathryn Nolan,Grocery FILL,BOK,Purchased Groceries
4,2023-01-02,Monday,Morning (7AM-12PM),Leah Santos,Grocery FILL,Mifflin,Purchased Groceries


In [7]:
import pandas as pd
from ipywidgets import interact, widgets
import plotly.express as px

# Convert 'Date' column to datetime
df_dates['Date'] = pd.to_datetime(df_dates['Date'])

# Extract month and year from 'Date' column
df_dates['YearMonth'] = df_dates['Date'].dt.to_period('M').astype(str)  

def plot_entries_per_month(fill_type):
    if fill_type == 'All Fill Types':
        filtered_df = df_dates.copy()  
    else:
        filtered_df = df_dates[df_dates['Fill Type'] == fill_type].copy()  

    filtered_df.loc[:, 'YearMonth'] = filtered_df['Date'].dt.to_period('M').astype(str)  

    monthly_counts = filtered_df.groupby('YearMonth').size().reset_index(name='Total Entries')

    # Set figure size
    fig = px.bar(monthly_counts, x='YearMonth', y='Total Entries', 
                 labels={'YearMonth': 'Month', 'Total Entries': 'Total Entries'},
                 title=f'Total Fills per Month for {fill_type}')
    fig.update_layout(xaxis_tickangle=-45, yaxis_gridcolor='lightgrey', 
                      width=800, height=500) 
    fig.show()

# Get unique values of 'Fill Type' for dropdown menu excluding 'nan'
fill_types = df_dates['Fill Type'].dropna().unique()
fill_types = list(fill_types)
fill_types.append('All Fill Types')  

# Create interactive dropdown menu
interact(plot_entries_per_month, fill_type=widgets.Dropdown(options=fill_types, description='Fill Type:'))


interactive(children=(Dropdown(description='Fill Type:', options=('Rescued Groceries', 'Purchased Groceries', …

<function __main__.plot_entries_per_month(fill_type)>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

def plot_donations_by_month(year, month):
    # Filter the DataFrame for the selected month and year
    monthly_data = df_dates.loc[(df_dates['Date'].dt.year == year) & (df_dates['Date'].dt.month == month)]
    
    # Group the data by date and time of day and count the number of donations for each group
    monthly_donations = monthly_data.groupby(['Date', 'Time of Day']).size().unstack(fill_value=0)
    
    # Plot the data
    plt.figure(figsize=(10, 6))
    monthly_donations.plot(kind='line', marker='o', linestyle='-')
    plt.title(f'Donations by Time of Day in {month}/{year}')
    plt.xlabel('Date')
    plt.ylabel('Number of Donations')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend(title='Time of Day')
    plt.tight_layout()
    # Show the figure
    plt.show()

# Define the options for the dropdown menu
years = [2023]  # Add more years if needed
months = range(1, 13)  # January to December

# Create interactive dropdown menus
interact(plot_donations_by_month, year=widgets.Dropdown(options=years, description='Year:'),
         month=widgets.Dropdown(options=months, description='Month:'))


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


interactive(children=(Dropdown(description='Year:', options=(2023,), value=2023), Dropdown(description='Month:…

<function __main__.plot_donations_by_month(year, month)>